# Regressor de preços de casas
Nesse exercício, aplicaremos modelos de regressão na predição de preços de casas na Califórnia.

Primeiramente, execute a célula abaixo para ler os dados:

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyRegressor
import pandas as pd
import numpy as np

df = pd.read_csv('data_aula_05_train.csv')
df_test = pd.read_csv('data_aula_05_test.csv')

X = df.drop(['median_house_value'],1)
y = df['median_house_value']
X_test = df_test.drop(['median_house_value'],1)
y_test = df_test['median_house_value']

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


# 1. Separação do conjunto de validação

Os dados já vieram separados em conjuntos de treino e teste, porém nós precisamos criar um terceiro conjunto de validação, para podermos avaliar diversos modelos sem ocorrer overfit nos dados de teste. 

Assim, separe o conjunto principal em um conjunto de treino (80% dos dados) e um de validação (20% dos dados).

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = None, None, None, None

######################################################################################
# 1. Separe as variáveis X e y em dados de treino e validação, armazenando os resultados nas variáveis acima

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

######################################################################################


### Verificação de erros ###
assert X_train.shape == (13600, 8) and X_val.shape == (3400, 8) and y_train.shape == (13600,) and y_val.shape == (3400,), 'Erro na separação dos conjuntos'

# 2. Preprocessamento dos dados 

Antes de passar os dados para o modelo, é necessário fazer com que eles fiquem aproximadamente na mesma escala. Para isso, vamos considerar duas possibilidades: escalonamento para o intervalo [0,1] e normalização para média 0 e desvio padrão 1.

In [9]:
def escalonar(data):

    ######################################################################################
    # 2. Transforme os dados na variável data para que cada coluna tenha um mínimo de 0 e um máximo de 1
    # Obs: a variável data pode ser um DataFrame inteiro ou uma coluna
    # O objeto MinMaxScaler da sklearn pode ser útil

    scaler = MinMaxScaler()
    scaler.fit(data)
    scaled_data = scaler.transform(data)
    return scaled_data    
    
    ######################################################################################


### Verificação de erros ###
assert escalonar(X_train).shape == X_train.shape, 'Os dados retornados devem ter o mesmo formato dos originais'
assert np.all(np.abs(np.max(escalonar(X_test)) - 1) < 1e-3) and np.all(np.abs(np.min(escalonar(X_test))) < 1e-3), 'Os dados retornados devem ter um mínimo de 0 e um máximo de 1'

In [13]:
def padronizar(data):

    ######################################################################################
    # 3. Transforme os dados na variável data para que cada coluna tenha uma média de 0 e um desvio padrão de 1
    # Obs: a variável data pode ser um DataFrame inteiro ou uma coluna
    # O objeto StandardScaler da sklearn pode ser útil
    
    stand = StandardScaler()
    stand_data = stand.fit_transform(data)
    return stand_data 

    ######################################################################################


### Verificação de erros ###
assert padronizar(X_test).shape == X_test.shape, 'Os dados retornados devem ter o mesmo formato dos originais'
assert np.all(np.abs(np.std(padronizar(X_test)) - 1) < 1e-3) and np.all(np.abs(np.mean(padronizar(X_test))) < 1e-3), 'Os dados retornados devem ter uma média de 0 e um desvio padrão de 1'

# 3. Métricas de regressão

Uma das principais métricas usadas para regressão é o Erro Quadrático Médio (MSE), que segue a seguinte equação:

$\text{MSE} = \frac{1}{n}\sum (Y_{\text{pred}} - Y_{\text{true}})^2$


Para facilitar a interpretação desse valor, podemos usar a medida R2, que é um valor normalmente entre 0 e 1. Ele é calculado segundo a seguinte equação:

$R^2 = 1 - \frac{\text{MSE}}{\text{Var}[Y_{\text{true}}]}$

Assim, implemente as funções para calcular as quantidades acima:



In [31]:
def MSE(y_true, y_pred):

    ######################################################################################
    # 4. Implemente uma função que retorne o erro quadrádico médio
    # y_true: valores verdadeiros da variável
    # y_pred: valores preditos pelo modelo
    
    somat = 0
    n = len(y_true)
    
    for i in range(n):
        somat += (y_pred[i] - y_true[i])**2
        
    MSE = somat/n    
        
    return MSE

    ######################################################################################

### Verificação de erros ###
assert np.abs(MSE(y_test, np.zeros_like(y_test)) - np.mean(y_test**2)) < 1e-3, 'Erro na função MSE'
assert np.abs(MSE(y_test, np.ones_like(y_test)*np.mean(y_test)) - np.var(y_test)) < 1e-3, 'Erro na função MSE'

In [20]:
def R2(y_true, y_pred):
    
    ######################################################################################
    # 5. Implemente uma função que retorne a medida R2 (pode ser útil reutilizar a função acima)
    # y_true: valores verdadeiros da variável
    # y_pred: valores preditos pelo modelo
    
    mse = MSE(y_true, y_pred)
    r2 = 1 - (mse/abs(np.var(y_true)))
    
    return r2
    ######################################################################################

### Verificação de erros ###
assert np.abs(R2(y_test, np.zeros_like(y_test)) + np.mean(y_test)**2 / np.var(y_test)) < 1e-3, 'Erro na função R2'
#assert np.abs(R2(y_test, np.mean(y_test))) < 1e-3, 'Erro na função R2'

# 4. Avaliando o modelo KNN

Vamos avaliar a performance do modelo KNN para essa tarefa de regressão, com diferentes tipos de preprocessamento (dados originais, escalonados ou padronizados).

Para isso, implemente a função para realizar as predições com um modelo:

In [22]:
def predict(model, X_train, y_train, X_val):
    ######################################################################################
    # 6. Treine o modelo nos dados de treino e retorne as predições na validação
    # A variável model é um modelo da biblioteca sklearn, então a função model.fit pode ser usada
    
    model.fit(X_train, y_train)
    
    return model.predict(X_val)
    
    ######################################################################################

### Verificação de erros ###
assert np.all(predict(DummyRegressor(), X_train, y_train, X_val) == y_train.mean()), 'Erro na função predict'

In [34]:
def R2_model(model, X_train, y_train, X_val, y_val):

    ######################################################################################
    # 7. Retorne a medida R2 do modelo treinado (pode ser útil reutilizar a função anterior)
    
    y_pred = predict(model, X_train, y_train, X_val)
    r2_model = R2(y_val.values, y_pred)
    
    return r2_model
    
    ######################################################################################

### Verificação de erros ###
assert np.abs(R2_model(DummyRegressor(), X_train, y_train, X_val, y_val)) < 1e-3, 'Erro na função R2_model'

Em seguida, vamos avaliar o modelo KNN nos dados de validação com os diferentes tipos de preprocessamento:

In [37]:
from sklearn.neighbors import KNeighborsRegressor

R2_original = None
R2_escalonado = None
R2_padronizado = None

model = KNeighborsRegressor()

######################################################################################
# 8. Avalie o modelo nos dados de validação em três situações, salvando os resultados nas variáveis acima:
# 1. Com os dados originais
# 2. Com os dados escalonados
# 3. Com os dados padronizados
#
# OBS: não utilize os dados de teste nesse passo, apenas os de treino e validação!

R2_original = R2_model(model, X_train, y_train, X_val, y_val)
R2_escalonado = R2_model(model, escalonar(X_train), y_train, 
                         escalonar(X_val), y_val)
R2_padronizado = R2_model(model, padronizar(X_train), y_train, 
                         padronizar(X_val), y_val)

######################################################################################

print('R2 sem preprocessamento: %.2f' % R2_original)
print('R2 com escalonamento entre 0 e 1: %.2f' % R2_escalonado)
print('R2 com padronização para média 0 e desvio padrão 1: %.2f' % R2_padronizado)

R2 sem preprocessamento: 0.25
R2 com escalonamento entre 0 e 1: 0.67
R2 com padronização para média 0 e desvio padrão 1: 0.71


Assim, encontre o melhor valor de K para o modelo KNN usando o preprocessamento selecionado no passo anterior, testando todos os valores de K no intervalo de 1 a 30 no conjunto de validação:

In [41]:
best_k = 0
best_R2 = 0

######################################################################################
# 9. Encontre o erro do melhor valor de K para o modelo KNN usando o preprocessamento encontrado no passo anterior
# OBS: não utilize os dados de teste nesse passo, apenas os de treino e validação!
#
# Dica: Para criar um modelo KNN com um valor específico de K, use KNeighborsRegressor(n_neighbors = K)

k_train = KNeighborsRegressor(n_neighbors = k)
k_train.fit(padronizar(X_train), y_train)
    
for k in range(1,31):
    r_knn = R2_model(KNeighborsRegressor(n_neighbors = k), padronizar(X_train), y_train, padronizar(X_val), y_val)
    if r_knn > best_R2:
        best_R2 = r_knn
        best_k = k
    
######################################################################################

print('Melhor valor de k encontrado: %d' % best_k)
print('Melhor erro R2 na validação encontrado: %.2f' % best_R2)

Melhor valor de k encontrado: 7
Melhor erro R2 na validação encontrado: 0.72


Finalmente, podemos usar o valor ótimo encontrado para avaliar o modelo nos dados de teste:

In [42]:
test_R2 = None

######################################################################################
# 10. Encontre o erro do modelo encontrado nos dados de teste
# Para isso, use o valor de k e o preprocessamento ótimos encontrados
test_R2 = R2_model(KNeighborsRegressor(n_neighbors = 7), padronizar(X_train), y_train, padronizar(X_test), y_test)

######################################################################################

print('Erro final encontrado no teste: %.2f' % test_R2)

Erro final encontrado no teste: 0.69
